In [107]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torchinfo import summary
from torchvision.models import vgg19

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
# import tensorflow as tf
import torch.nn as nn
from math import sqrt
import pandas as pd
import numpy as np
import random
import torch
import os
# import imshow

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')


Running on the GPU


In [91]:
# Data = np.load("../data/Dataset.npy",allow_pickle=True)
X = np.load("../data/Images.npy",allow_pickle=True)
y= np.load("../data/Labels.npy",allow_pickle=True)
X.shape , y.shape , X.dtype , y.dtype

((130, 3, 224, 224), (130, 19), dtype('int64'), dtype('int64'))

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape    

((104, 3, 224, 224), (26, 3, 224, 224), (104, 19), (26, 19))

In [96]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [28]:
model = vgg19(weights='IMAGENET1K_V1')

In [82]:
# VGG model for feature extraction
model = vgg19(weights='IMAGENET1K_V1')

In [137]:
class VGG_model(nn.Module):
    def __init__(self,input_dim ,output_dim=1,FC=[1024,512]):
        super().__init__()
        self.Linear1 = nn.Linear(in_features=input_dim,out_features = FC[0])
        self.Linear2 = nn.Linear(in_features = FC[0], out_features = FC[1])
        self.Linear3 = nn.Linear(in_features = FC[1], out_features = output_dim)
    
    def forward(self, X):
        X = self.Linear1(X)
        X = nn.functional.relu(X)
        X = self.Linear2(X)
        X = nn.functional.relu(X)
        X = self.Linear3(X)
        X = nn.functional.softmax(X, dim=1)
        return X
        

VGG_clf = VGG_model(input_dim =25088,output_dim=19, FC=[128,512])
summary(VGG_clf, input_size=(1,25088))


Layer (type:depth-idx)                   Output Shape              Param #
VGG_model                                [1, 19]                   --
├─Linear: 1-1                            [1, 128]                  3,211,392
├─Linear: 1-2                            [1, 512]                  66,048
├─Linear: 1-3                            [1, 19]                   9,747
Total params: 3,287,187
Trainable params: 3,287,187
Non-trainable params: 0
Total mult-adds (M): 3.29
Input size (MB): 0.10
Forward/backward pass size (MB): 0.01
Params size (MB): 13.15
Estimated Total Size (MB): 13.25

In [139]:
# training the model for face recognition

n_epochs = 10
batch_size = 5
lr = 0.3

VGG_featurizer = vgg19(weights='IMAGENET1K_V1')
VGG_featurizer.trainable = False
VGG_clf = VGG_model(input_dim =25088,output_dim=19, FC=[1024,512])
VGG_featurizer.to(device)
VGG_clf.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(VGG_clf.parameters(), lr=lr)

train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

train_losses = []
test_losses = []
train_acc = []
test_acc = []

for epoch in range(n_epochs):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        features = VGG_featurizer.features(data)
        features = features.view(features.size(0), -1)
        # print(features.shape)
        output = VGG_clf(features)

        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        # pred = output.argmax(dim=1, keepdim=True)

    print("Epoch : {} , Train loss : {}".format(epoch,loss.item()))



Epoch : 0 , Train loss : 2.7810165882110596
Epoch : 1 , Train loss : 2.7810165882110596
Epoch : 2 , Train loss : 3.0310165882110596
Epoch : 3 , Train loss : 3.0310165882110596
Epoch : 4 , Train loss : 2.7810165882110596
Epoch : 5 , Train loss : 2.7810165882110596
Epoch : 6 , Train loss : 3.0310165882110596
Epoch : 7 , Train loss : 3.0310165882110596
Epoch : 8 , Train loss : 2.7810165882110596
Epoch : 9 , Train loss : 2.7810165882110596


In [140]:
y_f = VGG_featurizer.features(X_test.to(device))
y_f = y_f.view(y_f.size(0), -1)
y_hat = VGG_clf(y_f)
out = torch.argmax(y_hat, dim=1)
out


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1], device='cuda:0')